In [2]:
import numpy as np
from astropy.cosmology import Planck15 as cosmo
import h5py

# adding path to GPR_for_IM directory in order to import relevant scripts
import sys
sys.path.append('../')
import obs_tools as obs

Frequency range of data:

In [5]:
df = 1 # frequency resolution of data (MHz)
vmin = 899 # min frequency (MHz)
vmax = 1184 # max frequency (MHz)
freqs = np.arange(vmin, vmax, df)

Load unsmoothed data:

In [7]:
gsync = np.load('../Data/dT_sync_Stripe82_noBeam.npy')
free = np.load('../Data/dT_free_Stripe82_noBeam.npy')
psource = np.load('../Data/dT_psource_Stripe82_noBeam.npy')
pleak = np.load('../Data/dT_pleak_Stripe82_noBeam.npy')
FGnopol_data = gsync + free + psource
FGwpol_data = gsync + free + psource + pleak
HI_data = np.load('../Data/T_HI-MDSAGE_z_0.39.npy')
HI_data = np.swapaxes(HI_data, 1, 2)
noise_data = np.load('../Data/dT_noise.npy')

Smooth data by telescope beam:

In [8]:
# smoothing maps
lx,ly,lz = 1000,1000, 924.78 #Mpc/h
zmax = 0.58
sigma_beam = 1.55
FGnopol_HI_noise_smoothed = obs.ConvolveCube(np.copy(FGnopol_data+HI_data+noise_data),zmax,lx,ly,sigma_beam, cosmo)
FGwpol_HI_noise_smoothed = obs.ConvolveCube(np.copy(FGwpol_data+HI_data+noise_data),zmax,lx,ly, sigma_beam, cosmo)
FGnopol_smoothed = obs.ConvolveCube(np.copy(FGnopol_data),zmax,lx,ly,sigma_beam, cosmo)
FGwpol_smoothed = obs.ConvolveCube(np.copy(FGwpol_data),zmax,lx,ly,sigma_beam, cosmo)
HI_noise_smoothed = obs.ConvolveCube(np.copy(HI_data+noise_data),zmax,lx,ly,sigma_beam, cosmo)
HI_smoothed = obs.ConvolveCube(np.copy(HI_data),zmax,lx,ly,sigma_beam, cosmo)
noise_smoothed = obs.ConvolveCube(np.copy(noise_data),zmax,lx,ly,sigma_beam, cosmo)
gsync_smoothed = obs.ConvolveCube(np.copy(gsync),zmax,lx,ly,sigma_beam, cosmo)
free_smoothed = obs.ConvolveCube(np.copy(free),zmax,lx,ly,sigma_beam, cosmo)
psource_smoothed = obs.ConvolveCube(np.copy(psource),zmax,lx,ly,sigma_beam, cosmo)
pol_smoothed = obs.ConvolveCube(np.copy(pleak),zmax,lx,ly,sigma_beam, cosmo)

Initialise file:

In [9]:
hf = h5py.File('data.h5', 'w')

Create two groups (kind of like "folders" within the data file), one for data that has beem smoothed with the telescope beam and one for unsmoothed data:

In [10]:
hf.create_dataset('freqs', data=freqs)

<HDF5 dataset "freqs": shape (285,), type "<i8">

In [11]:
g1 = hf.create_group('beam')
g2 = hf.create_group('nobeam')

No beam:

In [12]:
# foregrounds folder:
g3 = hf.create_group('nobeam/foregrounds')
g3.create_dataset('sync',data=gsync)
g3.create_dataset('free',data=free)
g3.create_dataset('psource',data=psource)
g3.create_dataset('pleak',data=pleak)
# main folder:
g2.create_dataset('HI',data=HI_data)
g2.create_dataset('noise',data=noise_data)

<HDF5 dataset "noise": shape (256, 256, 285), type "<f8">

With beam:

In [13]:
# foregrounds folder:
g4 = hf.create_group('beam/foregrounds')
g4.create_dataset('sync',data=gsync_smoothed)
g4.create_dataset('free',data=free_smoothed)
g4.create_dataset('psource',data=psource_smoothed)
g4.create_dataset('pleak',data=pol_smoothed)
g4.create_dataset('nopol',data=FGnopol_smoothed)
g4.create_dataset('wpol',data=FGwpol_smoothed)
# main folder:
g1.create_dataset('HI+noise',data=HI_noise_smoothed)
g1.create_dataset('noise',data=noise_smoothed)
g1.create_dataset('HI',data=HI_smoothed)
g1.create_dataset('FGwpol+HI+noise',data=FGwpol_HI_noise_smoothed)
g1.create_dataset('FGnopol+HI+noise',data=FGnopol_HI_noise_smoothed)

<HDF5 dataset "FGnopol+HI+noise": shape (256, 256, 285), type "<f8">

In [14]:
hf.close()